# Creating a Horror Book Database from 'lubimyczytac.pl'
#### Part 1 of **Machine Learning Horror** project

*Lubimyczytać.pl* is a popular Polish online platform focused on literature. It allows users to share their book reviews, read the opinions of other readers, and create lists of books they've read or plan to read.\
This service is a meeting place for book enthusiasts where they can exchange their literary experiences and recommendations.

From a young age, I had a passion for reading horror books, and it seems that I've even read a significant portion of popular books in this genre multiple times.\
The *lubimyczytac.pl* website is an excellent literature resource, but I always spent long hours jumping from page to page in search of the right book for me. Therefore, I decided to simplify this task.\
\
I used web scraping to gather data (title, authors, user portal ratings, the number of ratings, and the number of readers, which I understand as the popularity of a given book) from this website, regarding books with the "horror" tag.\
Additionally, the materials were supplemented with short summaries of each book. I chose to organize all this data in the form of a MySQL database so that I could quickly select the books that interest me using SQL.

In [168]:
#packages

import pandas as pd
import requests
from bs4 import BeautifulSoup

import sqlalchemy
from sqlalchemy import create_engine

In [91]:
#building request books with horror tag, published in 2010-2022.
#creating empty lists for scrapped data
title, author, num_of_readers, score, num_of_scores, link= [],[],[],[],[],[]
#setting last page, start and end-year for range loop
last_page = 58
start_year = 2010
end_year = 2022
#iterate through pages
for page in range(1,last_page):
    base_url = f'https://lubimyczytac.pl/katalog?page={page}&listId=booksFilteredList&tags[]=horror&rating[]=0&rating[]=10&publishedYear[]={start_year}&publishedYear[]={end_year}&catalogSortBy=published-desc&paginatorType=Standard'
    r = requests.get(base_url)
    soup = BeautifulSoup(r.content)
    #Titles
    titles = soup.find_all('a', class_='authorAllBooks__singleTextTitle')
    for t in titles:
        t=t.get_text('|', strip=True) #I've decided to use get_text from bs4. In code chunks it looks just better and simplier than using regex replacement for all html keywords.
        title.append(t)
    #Authors
    authors = soup.find_all('div', class_ = 'authorAllBooks__singleTextAuthor authorAllBooks__singleTextAuthor--bottomMore')
    for a in authors:
        a = a.get_text('|', strip = True).replace('|,|', ', ') #when book has more than one author, get_text produce string with '|,|' between authors. We can just simply replace it with ', '.
        author.append(a)
    #Num_of_readers
    readers = soup.find_all('span', class_ = 'small grey mr-2 mb-3')
    for r in readers:
        r = r.get_text('|', strip=True).split()[1]
        num_of_readers.append(int(r))
    #Num_of_scores
    num_scores = soup.find_all('div', class_ = 'listLibrary__ratingAll')
    for ns in num_scores:
        ns = ns.get_text('|', strip = True)
        if ns == 'ocen': #in a html script when book has no scores, the 0 number is not presented. I've decided to check it in a loop and 
            ns = 0
        else:
            ns = ns.split()[0]
        num_of_scores.append(int(ns))
    #Scores
    scores = soup.find_all('span', class_='listLibrary__ratingStarsNumber')
    for s in scores: 
        s = s.get_text('|', strip = True).replace(',', '.')
        score.append(float(s))
    #Links
    links = soup.find_all('a', class_ = 'authorAllBooks__singleTextTitle float-left')
    for l in links:
        l = str(l)
        l = l.replace('<a class="authorAllBooks__singleTextTitle float-left" href="/ksiazka/', '').split('">')[0] #This time i've used replace and split from string lib, couse I need here a specific part of link.
        link.append(l)
#creating df with scrapped data
books_table = pd.DataFrame(list(zip(title, author, score, num_of_scores, num_of_readers, link)), columns = ['title','author', 'score', 'number_of_scores', 'number_of_readers', 'link'])
books_table["book_id"] = books_table.index + 1 #adding 'book_id" index - primary key for books
books_table = books_table[books_table.columns[[6,0,1,2,3,4,5]]] #reordering cols

In [ ]:
#scrapping descritption table
base_url = 'https://lubimyczytac.pl/ksiazka/'
desc = []
for l in books_table['link']:
    req = requests.get(base_url + l)
    soup = BeautifulSoup(req.content)
    description = soup.find('div', class_ = 'collapse-content')
    description = description.get_text('|', strip = True).replace('|', ' ')
    desc.append(description)
books_description = pd.DataFrame(list(zip(books_table['book_id'], desc)), columns = ['book_id', 'description'])

In [195]:
con_string = 'mysql+pymysql://root:@localhost:3306/lubimyczytac' #creating connection with db by sqlalchemy
engine = create_engine(con_string)
conn = engine.connect()

In [196]:
# uploading pandas tables to db
books_table.to_sql('book_table', conn, if_exists='replace', index = False)
books_description.to_sql('book_description', conn, if_exists='replace', index = False)

1710

In [215]:
#100 horror books with score greater than 5 and with min. 500 readers - sort by score in descending order
query1 = 'SELECT bk.title "Title",\
                bk.author "Author/s",\
                bk.score "Score",\
                bk.number_of_readers "Number of readers",\
                bd.description "Book description"\
          FROM book_table bk\
          LEFT JOIN book_description bd ON bk.book_id=bd.book_id\
          WHERE bk.score > 5 AND bk.number_of_readers > 300\
          ORDER BY score desc\
          LIMIT 100'
query1_df = pd.read_sql(query1, conn)
query1_df

,Title,Author/s,Score,Number of readers,Book description
0,Tokyo Ghoul tom 14,Sui Ishida,8.4,363,„Chciałbym usłyszeć twoją historię”. Operacja ...
1,Atak Tytanów #12,Isayama Hajime,8.3,342,Po szokującym ujawnieniu tożsamości tytana kol...
2,Zgroza w Dunwich i inne przerażające opowieści,H.P. Lovecraft,8.3,7047,"Jeśli ist­niała rzecz, któ­rej Love­craft nie ..."
3,Atak Tytanów #7,Isayama Hajime,8.2,491,"By poznać „sekret tytanów”, drzemiący w piwnic..."
4,Wilczy zew,Paweł Leśniak,8.2,671,Świat umarłych stoi przed tobą otworem… Max do...
...,...,...,...,...,...
95,Eksperyment,Grzegorz Kopiec,7.0,845,"Co roku, wraz z nastaniem jesieni na całym świ..."
96,Judge 1,Yoshiki Tonogai,7.0,321,Kłamstwo Hiro przypadkowo prowadzi do śmierci ...
97,Harry Angel,William Hjortsberg,7.0,1233,"Koniec lat 50., Nowy Jork, niejaki Louis Cyphr..."
98,Żywiołaki,Michael McDowell,7.0,498,„Matki Savage’ów pożerają własne dzieci!” W Al...


In [217]:
#10 most popular horror books
query2 = 'SELECT bk.title "Title",\
                 bk.author "Author/s",\
                 bk.number_of_readers "Number of readers"\
          FROM book_table bk\
          ORDER BY bk.number_of_readers desc\
          LIMIT 10'
query2_df = pd.read_sql(query2, conn)
query2_df

,Title,Author/s,Number of readers
0,Smętarz dla zwierzaków,Stephen King,36588
1,Carrie,Stephen King,30203
2,Książę Mgły,Carlos Ruiz Zafón,22078
3,Joyland,Stephen King,18977
4,Drakula,Bram Stoker,17638
5,Bezsenność,Stephen King,14275
6,Dziewczyna z sąsiedztwa,Jack Ketchum,11557
7,Bazar złych snów,Stephen King,11512
8,Instytut,Stephen King,10809
9,Domofon,Zygmunt Miłoszewski,9808
